# `csv_to_sql`

Given a csv file

Return the sql INSERT statements

In [1]:
import pandas as pd
from os.path import join
from os import listdir
import numpy as np
import re

In [2]:
INPUT_DIR = join("..", "csv_inputs")
OUTPUT_DIR = join("..", "script_outputs")
HTML_OUTPUT_DIR = join("..", "html_outputs")

HTML_FILE_PATH = join(HTML_OUTPUT_DIR, "csv_to_sql.html")
SCRIPT_OUTPUT_PATH = join(OUTPUT_DIR, "courses_insert_statements.sql")

In [3]:
files = listdir(INPUT_DIR)
print("INPUT_DIR contents...", files)

INPUT_DIR contents... ['csai_scraped_data_response.csv']


In [4]:
INPUT_FILE = join(INPUT_DIR, files[0])

In [5]:
df = pd.read_csv(INPUT_FILE)

In [6]:
courses_df = df[df['type'] == 'course']
courses_df.head()

,club_course_name,advisor,advisor_email,advisor_phone,affiliation,box,concurrent,contact_email,contact_email_2,contact_person,...,office,office_hours,phone,prerequisites,recommended,research_interests,termsTypicallyOffered,type,types,units
93,CPE 100. Computer Engineering Orientation.,None,None,None,None,None,NaN,None,None,None,...,None,None,None,NaN,NaN,None,F,course,None,1
94,CPE 101. Fundamentals of Computer Science.,None,None,None,None,None,NaN,None,None,None,...,None,None,None,Appropriate Math Placement Level; or MATH 117 ...,NaN,None,"F, W, SP",course,None,4
95,CPE 105. Fundamentals of Computer Science I Su...,None,None,None,None,None,CPE/CSC 101.,None,None,None,...,None,None,None,NaN,NaN,None,TBD,course,None,1
96,CPE 108. Accelerated Introduction to Computer ...,None,None,None,None,None,NaN,None,None,None,...,None,None,None,MATH 118 (or equivalent) with a grade of C- or...,NaN,None,TBD,course,None,4
97,CPE 123. Introduction to Computing.,None,None,None,None,None,NaN,None,None,None,...,None,None,None,Basic computer literacy.,NaN,None,F,course,None,4


# What should the INSERT look like?

## for Courses

```mysql
INSERT INTO `Courses` 
(`courseId`, `dept`, `courseNum`, `termsOffered`, `units`, 
`courseName`, `concurrent`, `recommended`, `crossListedAs`, 
`standing`) 
VALUES 
(5,'DEPT',777,'SU',127,'HALF A DEGREEE','NA','NA','DEPT 666','NA');
```


## for Prerequisites

```mysql
INSERT INTO `Prerequisites` (`courseId`, `prereqCourse`)
VALUES (4, 'CSC 307 | CSC 308');
```

In [7]:
# What do our columns look like?
courses_df.columns

Index(['club_course_name', 'advisor', 'advisor_email', 'advisor_phone',
       'affiliation', 'box', 'concurrent', 'contact_email', 'contact_email_2',
       'contact_person', 'contact_phone', 'corequisites', 'courseName',
       'department', 'description', 'email', 'office', 'office_hours', 'phone',
       'prerequisites', 'recommended', 'research_interests',
       'termsTypicallyOffered', 'type', 'types', 'units'],
      dtype='object')

In [8]:
# Let's make a dataframe that filters for just the columns we want
mysql_df = courses_df[['club_course_name', 'termsTypicallyOffered', 'units', 'concurrent', 'recommended', 'prerequisites']]
mysql_df.columns = ['courseName', 'termsOffered', 'units', 'raw_concurrent_text', 'raw_recommended_text', 'raw_prerequisites_text']

In [9]:
# https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
pd.options.mode.chained_assignment = None  # default='warn'

In [10]:
mysql_df['crossListedAs'] = pd.DataFrame(["NA_CROSS_LIST"]*len(mysql_df))

In [11]:
# fillna
mysql_df['raw_prerequisites_text'] = mysql_df['raw_prerequisites_text'].fillna("NA_PREREQUISITE_TEXT")
mysql_df['raw_concurrent_text'] = mysql_df['raw_concurrent_text'].fillna("NA_CONCURRENT_TEXT")
mysql_df['raw_recommended_text'] = mysql_df['raw_recommended_text'].fillna("NA_RECOMMENDED_TEXT")

In [12]:
mysql_df['raw_standing_text'] = mysql_df['raw_prerequisites_text'].str.extract('(\w+ standing)', re.IGNORECASE)
# fillna
mysql_df['raw_standing_text'] = mysql_df['raw_standing_text'].fillna("NA_STANDING_TEXT")

def normalize_standing(s):
    s = str(s).lower()
    if s == "sophomore standing":
        return "SO"
    elif s == "junior standing":
        return "JR"
    elif s == "senior standing":
        return "SR"
    elif s == "graduate standing":
        return "GR"
    elif s == "na_standing_text":
        return "NA"
    else:
        print("unexpected s:", s, "... returning NA")
        return "NA"

mysql_df['standing'] = mysql_df['raw_standing_text'].apply(normalize_standing)

In [13]:
mysql_df['standing'].unique()  # explect just 'NA', 'SO', 'GR', 'JR'

array(['NA', 'SO', 'GR', 'JR'], dtype=object)

# uh oh we have some bad data `\xa0` let's fix that...
## which turns out to be a blessing in disguise because it helps us split the `dept` vs `courseNum`
## let's use `___` to delimit because `___` is not used anywhere else in the course names 
## we will change it back to a space later

In [14]:
mysql_df['courseName'].values[:5]

array(['CPE\xa0100. Computer Engineering Orientation.',
       'CPE\xa0101. Fundamentals of Computer Science.',
       'CPE\xa0105. Fundamentals of Computer Science I Supplemental Instruction.',
       'CPE\xa0108. Accelerated Introduction to Computer Science.',
       'CPE\xa0123. Introduction to Computing.'], dtype=object)

In [15]:
mysql_df['courseName'] = mysql_df['courseName'].str.replace(u'\xa0', "___")
mysql_df['courseName'].values[:5]

array(['CPE___100. Computer Engineering Orientation.',
       'CPE___101. Fundamentals of Computer Science.',
       'CPE___105. Fundamentals of Computer Science I Supplemental Instruction.',
       'CPE___108. Accelerated Introduction to Computer Science.',
       'CPE___123. Introduction to Computing.'], dtype=object)

# okay now extract the dept and courseNum

In [16]:
mysql_df['dept'] = mysql_df['courseName'].str.extract('(\w+(?=___))')
mysql_df['dept'].unique()

array(['CPE', 'CSC'], dtype=object)

In [17]:
mysql_df['courseNum'] = mysql_df['courseName'].str.extract('((?<=___)\d+)')
mysql_df['courseNum'].unique()  # interesting to see all the unique course numbers

array(['100', '101', '105', '108', '123', '133', '200', '202', '203',
       '233', '290', '315', '316', '321', '327', '328', '329', '333',
       '336', '350', '357', '367', '368', '400', '414', '416', '419',
       '422', '426', '428', '431', '432', '439', '441', '442', '446',
       '447', '450', '453', '454', '458', '461', '462', '464', '465',
       '469', '470', '471', '472', '476', '479', '482', '485', '488',
       '493', '494', '495', '515', '521', '522', '523', '532', '541',
       '542', '564', '569', '171', '209', '225', '231', '232', '234',
       '235', '236', '300', '301', '302', '303', '305', '307', '308',
       '309', '310', '311', '313', '320', '323', '325', '344', '348',
       '349', '365', '366', '369', '371', '377', '378', '402', '405',
       '406', '409', '410', '424', '429', '430', '435', '436', '437',
       '445', '448', '466', '468', '473', '474', '477', '478', '480',
       '481', '483', '484', '486', '487', '489', '490', '491', '492',
       '496', '497',

In [18]:
# change it back
mysql_df['courseName'] = mysql_df['courseName'].str.replace("___", " ")

# also `'F, W, SP'` is bad because our MySQL database uses `SET` which does not expect the whitespace between each element. 
## we want to change that to `'F,W,SP'`

# OH WOW! That's unexpected `TBD` is a termOffered... `¯\_(ツ)_/¯ `
```
array(['F', 'F,W,SP', 'TBD', 'F,W,SP,SU', 'F,SP', 'F,W', 'SP', 'W',
       'W,SP'], dtype=object)
In [20]:
```


In [19]:
mysql_df['termsOffered'] = mysql_df['termsOffered'].str.replace(' ','')
mysql_df['termsOffered'].unique()  # interesting to see all the unique terms offered

array(['F', 'F,W,SP', 'TBD', 'F,W,SP,SU', 'F,SP', 'F,W', 'SP', 'W',
       'W,SP'], dtype=object)

# wait let's check for bad data inside `units`

In [20]:
mysql_df['units'].unique()

array(['1', '4', '1-2', '1-4', '3', '2', '6', '12', '2-4'], dtype=object)

# Oh no! 
## What does `12` units mean? 
CSC 495 http://catalog.calpoly.edu/coursesaz/csc/
## How do we represent `1-2` units in a MySQL DB?
I've changed it to `VARCHAR(5)`

# one last bad data check
### darn.. how did `nan` get in there?

In [21]:
mysql_df['crossListedAs'].unique()

array(['NA_CROSS_LIST', nan], dtype=object)

In [22]:
mysql_df['crossListedAs'] = mysql_df['crossListedAs'].fillna('NA_CROSS_LIST')
mysql_df['crossListedAs'].unique()

array(['NA_CROSS_LIST'], dtype=object)

In [23]:
mysql_df.head()

,courseName,termsOffered,units,raw_concurrent_text,raw_recommended_text,raw_prerequisites_text,crossListedAs,raw_standing_text,standing,dept,courseNum
93,CPE 100. Computer Engineering Orientation.,F,1,NA_CONCURRENT_TEXT,NA_RECOMMENDED_TEXT,NA_PREREQUISITE_TEXT,NA_CROSS_LIST,NA_STANDING_TEXT,NA,CPE,100
94,CPE 101. Fundamentals of Computer Science.,"F,W,SP",4,NA_CONCURRENT_TEXT,NA_RECOMMENDED_TEXT,Appropriate Math Placement Level; or MATH 117 ...,NA_CROSS_LIST,NA_STANDING_TEXT,NA,CPE,101
95,CPE 105. Fundamentals of Computer Science I Su...,TBD,1,CPE/CSC 101.,NA_RECOMMENDED_TEXT,NA_PREREQUISITE_TEXT,NA_CROSS_LIST,NA_STANDING_TEXT,NA,CPE,105
96,CPE 108. Accelerated Introduction to Computer ...,TBD,4,NA_CONCURRENT_TEXT,NA_RECOMMENDED_TEXT,MATH 118 (or equivalent) with a grade of C- or...,NA_CROSS_LIST,NA_STANDING_TEXT,NA,CPE,108
97,CPE 123. Introduction to Computing.,F,4,NA_CONCURRENT_TEXT,NA_RECOMMENDED_TEXT,Basic computer literacy.,NA_CROSS_LIST,NA_STANDING_TEXT,NA,CPE,123


# Okay let's make the inserts
## **Notice** `courseId` is not needed because the database will _auto increment_ the `courseId`
## e.g.
```mysql
INSERT INTO `Courses` 
(`courseId`, `dept`, `courseNum`, `termsOffered`, `units`, 
`courseName`, `concurrent`, `recommended`, `crossListedAs`, 
`standing`) 
VALUES 
(5,'DEPT',777,'SU',127,'HALF A DEGREEE','NA','NA','DEPT 666','NA');
```

In [24]:
# https://stackoverflow.com/questions/31071952/generate-sql-statements-from-a-pandas-dataframe
def SQL_INSERT_STATEMENT_FROM_DATAFRAME(SOURCE, TARGET):
    sql_texts = []
    for index, row in SOURCE.iterrows():
        sql_texts.append('INSERT INTO `'+TARGET+'` (`'+ str('`, `'.join(SOURCE.columns))+ '`) VALUES '+ str(tuple(row.values)))
    return sql_texts

In [25]:
mysql_df.columns

Index(['courseName', 'termsOffered', 'units', 'raw_concurrent_text',
       'raw_recommended_text', 'raw_prerequisites_text', 'crossListedAs',
       'raw_standing_text', 'standing', 'dept', 'courseNum'],
      dtype='object')

# actually I modified the database a bit so our dataframe just simply fits...
```mysql
INSERT INTO `Courses` (`courseId`, `dept`, `courseNum`, `termsOffered`, `units`, `courseName`, `raw_concurrent_text`, `raw_recommended_text`, `raw_prerequisites_text`, `raw_standing_text`, `crossListedAs`, `standing`)
VALUES
	(2, 'CSC', 357, 'F,W,SP', 4, 'CSC 357. Systems Programming.', 'NA', 'NA', NULL, NULL, 'CPE 357', 'NA');
```

# Here are the INSERT statements

In [26]:
string_insert_statements = ";\n\n".join(SQL_INSERT_STATEMENT_FROM_DATAFRAME(mysql_df, "Courses"))
# one last semi-colon to satisfy my OCD
string_insert_statements = string_insert_statements + ";"
print(string_insert_statements)

INSERT INTO `Courses` (`courseName`, `termsOffered`, `units`, `raw_concurrent_text`, `raw_recommended_text`, `raw_prerequisites_text`, `crossListedAs`, `raw_standing_text`, `standing`, `dept`, `courseNum`) VALUES ('CPE 100. Computer Engineering Orientation.', 'F', '1', 'NA_CONCURRENT_TEXT', 'NA_RECOMMENDED_TEXT', 'NA_PREREQUISITE_TEXT', 'NA_CROSS_LIST', 'NA_STANDING_TEXT', 'NA', 'CPE', '100');

INSERT INTO `Courses` (`courseName`, `termsOffered`, `units`, `raw_concurrent_text`, `raw_recommended_text`, `raw_prerequisites_text`, `crossListedAs`, `raw_standing_text`, `standing`, `dept`, `courseNum`) VALUES ('CPE 101. Fundamentals of Computer Science.', 'F,W,SP', '4', 'NA_CONCURRENT_TEXT', 'NA_RECOMMENDED_TEXT', 'Appropriate Math Placement Level; or MATH 117 with a grade of C- or better; or MATH 118 with a grade of C- or better; or consent of instructor.', 'NA_CROSS_LIST', 'NA_STANDING_TEXT', 'NA', 'CPE', '101');

INSERT INTO `Courses` (`courseName`, `termsOffered`, `units`, `raw_concurren

# lastly let's structure the prequisites INSERTS

## e.g.
```mysql
INSERT INTO `Prerequisites` (`courseId`, `prereqCourse`)
VALUES (4, 'CSC 307 | CSC 308');
```

In [36]:
mysql_df[mysql_df['raw_prerequisites_text'].str.contains('and')]['raw_prerequisites_text'].unique()

array(['MATH 118 (or equivalent) with a grade of C- or better, significant experience in computer programming, and consent of instructor.',
       "An orientation course in student's major (EE 111 and EE 151; or CPE 100) and CPE/CSC 101.",
       'Open to undergraduate students and consent of instructor.',
       'CSC/CPE 102 and CSC/CPE 103, or CSC/CPE 202 and CSC/CPE 203; and one of the CSC 225, CPE/EE 229, or CPE/EE 233.',
       'CSC/CPE 102 and CSC/CPE 103 with a grade of C- or better or consent of instructor, or CSC/CPE 202 and CSC/CPE 203 with a grade of C- or better or consent of instructor; and CSC 225 or CPE/EE 229 or CPE/EE 233.',
       'CPE/EE 329 or CPE/EE 336 or both CPE 315 and CPE/CSC 357.',
       'CPE 315 or CPE 333; and CPE 233.',
       'EE 302 and EE 342; or CPE 327 and CPE 367.', 'EE 307 and EE 347.',
       'CSC/CPE 357, and CSC/CPE 225 or CPE/EE 229 or CPE/EE 233.',
       'CSC/CPE 453 and CPE 464.',
       'Sophomore standing and consent of instructor.',
     

# finally let's save

In [28]:
with open(SCRIPT_OUTPUT_PATH, 'w') as f:
    f.write(string_insert_statements)